# **Physics Engine**

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import random

from typing import Final
from matplotlib.patches import Polygon
from matplotlib import animation
from IPython.display import HTML
from math import sin, cos, radians, atan2, sqrt, pi

In [ ]:
x: Final[int] = 0
y: Final[int] = 1

interval: Final[int] = 10               # frame interval for animation
dt: Final[float] = interval / 1000      # (s), delta time
time = 0                                # time

g: Final[list] = np.array([0, -9.81])   # (m/s^2), gravity
e: Final[float] = 0.8                   # restitution
cpm: Final[float] = 0.04                # (1/m), air resistance

polygons = []
floor = []
normal = []
xlist = []
ylist = []
lxlist = []
lylist = []

In [ ]:
class PolyData:

    verteces = []           # verteces where the center of mass is the origon
    center = []             # x and y for center of mass position
    speed = 0               # (m/s)
    angle = 0               # angle in atan2 type degrees for velocity
    spin = 0                # (rad/s), negative for clockwise
    mass = 0                # (kg)
    velocity = []           # x & y component of center velocity
    inertia = 0             # moment of inertia for the polygon: https://www.omnicalculator.com/physics/mass-moment-of-inertia & https://www.omnicalculator.com/math/trigonometry
    collider = []           # collision vertex of the polygon
    colliderVelocity = []   # velocity of the collision vertex
    impulse = 0             # impulse of the collision

    def __init__(self, verteces, center, speed, angle, spin, mass, inertia):
        self.verteces = verteces
        self.center = center
        self.speed = speed
        self.angle = angle
        self.spin = spin
        self.mass = mass
        self.inertia = inertia
        self.velocity = np.array([
            self.speed * cos(radians(self.angle)),
            self.speed * sin(radians(self.angle))])

    def collision(self):
        global normal

        if (self.velocity[y] < 0 and (self.center[y] + self.verteces[:, y] < 0).any()):
            
            self.collider = self.verteces[np.argmin(self.verteces[:, y])]
            self.updateColliderVelocity()
            normal = np.array([0, 1])
            return True
        else:
            a = (floor[1][y] - floor[0][y]) / (floor[1][x] - floor[0][x])
            b = floor[0][y] - a * floor[0][x]
            
            for i in range(len(self.verteces)):
                vertex = self.center + self.verteces[i]

                if (vertex[y] < a * vertex[x] + b):
                    self.collider = self.verteces[i]
                    self.updateColliderVelocity()

                    absolute = sqrt((floor[1][x] - floor[0][x])**2 + (floor[1][y] - floor[0][y])**2)
                    normal = np.array([
                        -(floor[1][y] - floor[0][y]), 
                         (floor[1][x] - floor[0][x])]) / absolute

                    if (np.dot(self.colliderVelocity, normal) < 0):
                        return True

    def updateImpulse(self):
        self.impulse = -(1 + e) * ((np.dot(self.colliderVelocity, normal)) / ((1 / self.mass) + (np.cross(self.collider, normal)**2) / self.inertia))

    def updateColliderVelocity(self):
        self.colliderVelocity = np.array([self.velocity[x] - self.spin * self.collider[y], self.velocity[y] + self.spin * self.collider[x]])

    def updateVelocity(self):
        if (self.collision()):
            self.velocity += (self.impulse / self.mass) * normal
        self.velocity += g * dt

    def updateSpin(self):
        self.spin += (self.impulse / self.inertia) * np.cross(self.collider, normal)

    def updateSpeed(self):
        self.speed = sqrt(self.velocity[x]**2 + self.velocity[y]**2)

    def updateAngle(self):
        self.angle = atan2(self.velocity[y], self.velocity[x]) * 180/pi

    def updatePosition(self):
        self.center += self.velocity * dt

    def updateRotation(self):
        theta = self.spin * dt

        rotation = np.array((
            (np.cos(theta), -np.sin(theta)),
            (np.sin(theta),  np.cos(theta))))

        rows = range(len(self.verteces))

        for row in rows:
            self.verteces[row, :] = np.dot(rotation, self.verteces[row, :])

In [ ]:
fig, ax = plt.subplots(figsize=(4, 2), dpi=254)
plt.style.use('dark_background')
plt.xlim(0, 128)
plt.ylim(0, 64)
plt.title("x (m)", size=5)
plt.ylabel("y (m)", size=5)
plt.tick_params(axis='both', which='major', labelsize=5)

line, = ax.plot([], color='white', linewidth=0.75)
trace1, = ax.plot([], color='grey', linestyle='dotted', linewidth=0.5, alpha=1)
trace2, =  ax.plot([], color='grey', linestyle='dotted', linewidth=0.5, alpha=1)

floor = np.array([
    [ 64, 0], 
    [128, 32]])

square = PolyData(np.array([
    [-1., -1.],
    [ 1., -1.],
    [ 1.,  1.],
    [-1.,  1.]
]), np.array([1., random.uniform(5, 10)]), random.uniform(20, 30), random.uniform(35, 65), random.uniform(-10, 10), 100, 66.67)

triangle = PolyData(np.array([
    [-1., -1.],
    [ 1., -1.],
    [ 0.,  4.]
]), np.array([127., random.uniform(42, 47)]), random.uniform(20, 30), random.uniform(-35 - 90, -65 - 90), random.uniform(-10, 10), 100, 724.5)

polygons = [square, triangle]

squareVerteces = Polygon(square.verteces)
triangleVerteces = Polygon(triangle.verteces)
squareVerteces.set_color('c')
triangleVerteces.set_color('m')
ax.add_patch(squareVerteces)
ax.add_patch(triangleVerteces)


def init():
    line.set_data(floor[:, x], floor[:, y])

    squareVerteces.set_xy(square.verteces + square.center)
    triangleVerteces.set_xy(triangle.verteces + triangle.center)

    xlist.append(square.center[x])
    ylist.append(square.center[y])
    lxlist.append(triangle.center[x])
    lylist.append(triangle.center[y])

    return ax


def animate(i):
    global time
    time += dt

    for polygon in polygons:
        if (polygon.collision()):
            polygon.updateImpulse()
            polygon.updateSpin()
        polygon.updateVelocity()
        polygon.updatePosition()
        polygon.updateRotation()

    squareVerteces.set_xy(square.verteces + square.center)
    triangleVerteces.set_xy(triangle.verteces + triangle.center)

    xlist.append(square.center[x])
    ylist.append(square.center[y])
    lxlist.append(triangle.center[x])
    lylist.append(triangle.center[y])

    trace1.set_data(xlist, ylist)
    trace2.set_data(lxlist, lylist)

    return ax


anim = animation.FuncAnimation(
    fig, animate, frames=round(10000 / interval), interval=interval, init_func=init)

video = anim.to_html5_video()
html = HTML(video)
display(html)
plt.close()